In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [24]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 1 `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

### Our initial set of **occasions** for the category 1 bin is: 
1. `Drinking` - purpose to drink
    - `lunch drinking` - might need to further define these categories
    - `night drinking`
    - `after work drinking`
2. `Meal` - purpose is to eat
    - `lunch meal`
    - `after work meal`
    
We also said "relax alone" but not sure yet how it would look different.

### Something to think about:
- `Lunch` vs `dinner`
- `during sport events`
- `what type of foods or drinks to singles eat or drink`
- `what is their average dwell time at a table`

----------

In [112]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [167]:
hockey_path = '../data/hockey_3_text_processed.csv'

In [190]:
#silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [168]:
hockey = pd.read_csv(hockey_path)
#silvester = pd.read_csv(silvester_path)
#valentine = pd.read_csv(valentine_path)

In [191]:
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [171]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [172]:
fix_times(hockey)
#fix_times(valentine)
#fix_times(silvester)

In [192]:
fix_times(valentine)

In [193]:
valentine.columns

Index(['bar_id', 'order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'order_item_id', 'item_qty', 'sales_before_tax',
       'sales_inc_tax', 'guest_count', 'title', 'category_id', 'beer_brand_id',
       'beer_serving_type_id', 'beer_volume', 'total_orders_category_id_1.0',
       'total_orders_category_id_2.0', 'total_orders_category_id_3.0',
       'total_orders_category_id_4.0', 'total_orders_category_id_5.0',
       'total_orders_category_id_6.0', 'total_orders', 'order_day_of_week',
       'is_weekend', 'sharable', 'kids_meal', 'birthday', 'order_hour',
       'period_of_day', 'order_minute', 'order_item_minute', 'order_item_hour',
       'order_close_minute', 'order_close_hour', 'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step',
       'total_sales_before_tax', 'total_sales_inc_tax'],
      dtype='object')

Drop the `bar_id` feature, it's useless here.

In [120]:
hockey.columns

Index(['bar_id', 'order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'order_item_id', 'item_qty', 'sales_before_tax',
       'sales_inc_tax', 'guest_count', 'title', 'category_id', 'beer_brand_id',
       'beer_serving_type_id', 'beer_volume', 'total_orders_category_id_1.0',
       'total_orders_category_id_2.0', 'total_orders_category_id_3.0',
       'total_orders_category_id_4.0', 'total_orders_category_id_5.0',
       'total_orders_category_id_6.0', 'total_orders', 'order_day_of_week',
       'is_weekend', 'sharable', 'kids_meal', 'birthday', 'order_hour',
       'period_of_day', 'order_minute', 'order_item_minute', 'order_item_hour',
       'order_close_minute', 'order_close_hour', 'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step',
       'total_sales_before_tax', 'total_sales_inc_tax'],
      dtype='object')

In [174]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [ ]:
hockey = hockey[features]
#silvester = silvester[features]

In [195]:
valentine = valentine[features]

In [123]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,3,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [60]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [159]:
hockey_tables = create_tables(hockey)
#valentine_tables = create_tables(valentine)
#silvester_tables = create_tables(silvester)

In [196]:
valentine_tables = create_tables(valentine)

In [125]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  118664  tables in the Hockey data


In [197]:
print("We have ", len(valentine_tables), " tables in the Hockey data")

We have  40787  tables in the Hockey data


Keep only relevant features:

In [176]:
hockey.set_index('order_id', inplace=True)
#valentine.set_index('order_id', inplace=True)
#silvester.set_index('order_id', inplace=True)

In [198]:
valentine.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [103]:
COUNT = 10

In [51]:
import random

In [104]:
def get_tables_gc_1(tables):
    return tables[tables['guest_count'] == 1]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [127]:
hockey_1gc_tables = get_tables_gc_1(hockey_tables)
print("We have {} tables of a total of 1 guests".format(len(hockey_1gc_tables)))

We have 49664 tables of a total of 1 guests


In [128]:
#picked_hockey_tables = random.sample(list(hockey_1gc_tables.index), COUNT)

In [183]:
picked_hockey_tables = [512690383,
 521702519,
 521093892,
 517505343,
 521783372,
 521769692,
 512852707,
 525538989,
 514327163,
 520255453]

In [108]:
print("Got {} tables".format(len(picked_hockey_tables)))

Got 10 tables


## Table 1

In [162]:
#512690383
hockey[hockey.index == 512690383].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512690383,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:10:00,QUESA (VEG),2.0,1.225,1,7,17,0,0.0,Thursday,False,1,afternoon,0,0,0,20.43,1.0,5.95
512690383,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:16:00,FORBES MARG DBL,3.0,1.225,2,7,17,1,0.0,Thursday,False,1,afternoon,0,0,0,20.43,1.0,11.50
512690383,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:52:00,MINI DONUTS,2.0,1.225,3,7,17,10,0.0,Thursday,False,1,afternoon,1,0,0,20.43,0.5,2.98


**April 11th (Thursday), 17:10 - 18:23, a total of 1.2 hours, 3 meal steps**

**Meal Steps Description:**
1. Start
  - Meal (not very heavy or big)
      - Quesa
2. Still Start (6 minutes after)
  - Drink
      - Margarhita
3. Dessert, last order (~40+ minutes later)
  - Dessert
      - mini donutes
      
**Total drinks**: 1 cocktail, total meals: 1 main dish, 1 dessert

**`after work meal`**

`afternoon`, `small_meal`, `drink`

**Explanation**: Because of the time and the size of the meal, to me it seems like what a person after work would go and eat. Maybe even go to dinner after, but this seems like a "light" meal after work.

**Features that helped**: `type of meal` ("quesadia" is not really a main course), `price of meal` (5-6 $), and `time of meal` (5pm). 

## Table 2

In [163]:
hockey[hockey.index == 521702519].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_serving_type_id,beer_volume,total_orders,order_day_of_week,is_weekend,order_hour,period_of_day,order_item_minute,sharable,kids_meal,birthday,total_sales_before_tax,beer_brand_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,total_sales_inc_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:28:00,HENDRICKS,3.0,0.788333,1,7,11,0,0,0.0,2,Thursday,False,15,afternoon,28,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:47:00,HENDRICKS,3.0,0.788333,2,7,11,4,0,0.0,2,Thursday,False,15,afternoon,47,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91


**April 18th (Thursday), 15:28 - 16:15, total 0.7 hours, 2 meal steps**

**Meal Steps Description:**
1. Start
   - Drink
       - Gin
2. 20 Minutes Later:
   - Drink
       - Gin
       
**Total drinks**: 2 liquer drinks

**`Afternoon Drinking`**

`hard_liquer`, `afternoon`, `dwell_time`

**Explanation**: This is actually a very consistent person, he ordered two drinks exactly 20 minutes apart, and the table was closed ~20 minutes after the second drink.

**Features**: `time of meal` (3pm), `type of drink` (category 3, gin - hard liquer), `dwell time` (stayed ~40 min).

## Table 3

In [182]:
#521093892
hockey[hockey.index == 521093892].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 22:42:29,MOSCOW MULE DBL,3.0,2.356944,1,0,33,9,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 23:17:10,MOSCOW MULE DBL,3.0,2.356944,2,0,33,17,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:04:36,VODKA RED BULL,3.0,2.356944,3,0,33,28,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,9.50
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,INCEPTION BOMB,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,0.28,4.06
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,FIREBALL,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,5.95


**April 17th (Wednesday), 22:04 - 00:25, total of 2.35 hours, 4 meal steps.**

**Meal Steps Description:**
1. Start (40 minutes after sit down)
    - Drink
        - Moscow Mule (cocktail type drink)
2. 35 minutes after
    - Drink
        - Moscow Mule
3. 45 minutes after (1.15 hrs since person started drinking). (Notice the change to harsher liquer)
    - Drink
        - Vodka redbull (less of a sit down bar, more of a "dance" bar, or "Getting drunk bar", as opposed to previous drinks)
4. 20 minutes after that (~2 hours since started drinking)
    - Drink
        - Inception Bomb (another weird whisky shot)
        - Fireball (sweet whisky shot)
     (two for the road?)

**Total drinks: 2 cocktails, 1 mixed drink, 2 shots**

**`drinking`**

`late_night`, `hard_liquer`, `quantity of drinks`, `dwell_time`

**Explanation**: Its hard to say if this person was watching a game or something, or was ordering at a bar and sitting with a group. I would guess that the person is by themselves, since its a bit weird to keep ordering single drinks for yourself the whole night, even shots? Although at the end the person orders two whisky shots... hmm. Still I would say a single person drinking at night, could be more of a dance bar because of the type of drinks.
(its interesting also to note that the person started with classy drinks, and ended with whisky :P)

**Features**: `time of the order` and `closing time` (started at 10pm and ended past midnight), `quantity of drinks` (had a total of 5 drinks)

#### 4. Table 524421347

In [185]:
hockey[hockey.index == 524421347].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS.10PC,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,16.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,KIDPOP,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,JACKS.VEG.NACHOS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,21.26
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,KIDPOP,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,0.00
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,WINGS.10PC,2.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,1,0,0,89.35,1.0,16.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:31:04,POP.,4.0,1.144444,1,10,16,0,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,3.99
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,2019-04-21 17:37:39,STRW.MINTSODA,3.0,1.144444,2,10,16,1,0.0,Sunday,True,3,afternoon,0,0,0,89.35,1.0,8.56


** Mistake in the category **

We can add a rule for this

Notice there are only 3 meal steps but 7 food type items.
There is also a "kid" meal in here.

Here we can filter per meal type, there shouldn't be more than MAX 3 food items in one meal step. (if not total).

#### 5. Table 521095118

In [186]:
hockey[hockey.index == 521783372].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,SMOKIN.SALMON,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,23.96
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,Salmon.Rice/Veg,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,0.00


**April 18th (Thursday), 21:01 - 21:20, total of 0.3 hours, and 1 meal steps.**

**Meal Steps Description:**
1. Start 
    - Dinner (classic)
        - Salmon
        
**Total meals: 1 large/dinner meal**

**`dinner`**

`dwell_time`, `food_category_only`, `dinner_time`.

**Explanation**: Not much to explain, very classic single person dinner.


#### 6. Table 522574215

In [187]:
hockey[hockey.index == 521769692].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 18:41:00,LRG BONESHAKER,1.0,2.056667,1,19,29,0,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:24:00,BUF CHIC SALAD,2.0,2.056667,2,19,29,10,0.000,Thursday,False,1,dinner,0,0,0,29.5,1.0,15.00
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:25:00,LRG BONESHAKER,1.0,2.056667,2,19,29,10,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25


**April 18th (Thursday), 18:41 - 20:44, total of 2 hours, 2 meal steps.**

**Meal Steps Description:**

1. Start:
   - Drink
       - Half a liter of beer (ordered when sat down)
2. ~40 minutes after
   - Drink
       - Half a liter of beer (ordered when sat down)
   - Meal
       - Salad

**total drinks**: 2 half liter beers, total meals: 1 salad

**`casual drink & meal`** - (I guess similar to relaxing alone ?)

`dwell_time`, `beer`, `time_between_drinks`, `dinner_time`

**Explanation**: I didn't want to say casual dinner, because the person first ordered a beer, and drank the beer for 40 minutes. I would say that he was watching a game or something, and after 40 minutes (maybe half time?) ordered food and another drink. Especially since he then goes on to sit another 1.2 hours with a beer and salad.. I mean no one takes that long to eat a salad unless they're donig someting else.

**Features**:  The fact that there was a long period of time for two beers. The time it took between beers, the time of the table, dinner time, and the light meal.



#### 7. Table 522779155

In [188]:
hockey[hockey.index == 512852707].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-11 23:40:03,GUINNESS.Pt,1.0,1.353889,1,0,20,0,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 00:41:03,GUINNESS.Pt,1.0,1.353889,2,0,20,15,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,$Sub.Swt.Pot.Fry-,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,0,0,0,27.57,0.5,1.49
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,FINGERS.SAUCED,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,1,0,0,27.57,0.5,8.44


**April 11th (Thursday), 23:40 - 01:01, total of 1.35 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Guinnes, half a liter
2. 1hr later
    - drink
        - Guiness, half a liter
3. 20 minutes later
    - meal
        - chicken fingers, and sweet potato fries
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

`late casual drink`

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

#### 8. Table 515363895

In [189]:
hockey[hockey.index == 525538989].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 17:44:35,KEITHS.XL,1.0,1.696389,1,9,23,0,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:27:36,KEITHS.XL,1.0,1.696389,2,9,23,10,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:46:51,CHEESE.CHIKN.MAC,2.0,1.696389,3,9,23,14,0.000,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,19.97


**April 23rd (Tuesday), 17:44 - 19:25, total of 1.7 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Keiths, half a liter
2. 45 min later
    - drink
        - Keiths, half a liter
3. 20 minutes later
    - meal
        - chicken mac and cheese (bar food.. amarican bar food. I miss mac&cheese)
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

**`casual drink & meal`**

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [199]:
valentine_1gc = get_tables_gc_1(valentine_tables)
print("We have {} tables of a total of 1 guests".format(len(valentine_1gc)))

We have 15485 tables of a total of 1 guests


In [200]:
picked_valentine_tables = random.sample(list(valentine_1gc.index), COUNT)

In [204]:
picked_valentine_tables = [434780854,
 434753224,
 447023765,
 434766693,
 446799837]

## Table 1

In [206]:
valentine[valentine.index == 434780854].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,BEEFEATER**,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,MARTINI,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.97
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,WATER.,4.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD THAI,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,BEEFEATER**,3.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD.THAI.CHICKN,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,20.74
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:10:00,BAR.MANGER.Pt,1.0,3.084167,3,9,44,13,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.72
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:44:00,RICKARD.RED.Pt,1.0,3.084167,4,9,44,21,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 16:40:00,RICKARD.RED.Pt,1.0,3.084167,5,9,44,35,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43


**Feb 10th (Sunday), 14:13 - 17:18, total of ~3 hour, 5 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - martini
        - Beefeater ??? -cheap, maybe shot?
        - water
2. 15 min later
    - meal
        - pad thai chicken
    - drink
        - beefeater ???
3. 40 min later
    - drink
        - "bad manager" beer, half liter
      
4. 35 min later
    - drink
        - red pt, half liter
5. 1 hour later
    - drink
        - red pt, hald liter
        

**total drinks**: 4 or 6, depending if we cound beefeater, **total meals**: 1 meal.

**`casual afternoon drink & meal`**

`dwell_time`, `amount of alcohol`, `time between drinks`

**Explanation**: This one was not easy. 3 whole hours at a bar. at first it looks like a mistake, but then you see that there are actual steps, where you start with a drink, eat a meal, and finish with two more drinks. Hard to say what type of occasion this is, but I would say a casual meal and drink, since there is a meal consumed, and a good amount of drink and beer consumed.

- maybe because of the martini and amount of total alcohol its not casual?.. but its 2pm :/


#### 2. Table 435005397

In [207]:
valentine[valentine.index == 434753224].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS..,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,0.00
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS.5PC,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,10.47


**Feb 10th (Sunday), 21:56 - 22:25, total of 0.5 hours (30 minutes), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - chicken fingers
        
**total meals**: 1 late night sharable meal

**`late night munch`** - a sharbale item late at night

`sharable`, `late_night`, `1_step/1 item`


**Explanation**: Looks like a late munch. I would even say maybe its a take out, then again it could be a person stopped for wings and then left.



#### 3. Table 435011399

In [211]:
valentine[valentine.index == 447023765].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:08:00,POP.,4.0,0.985556,1,4,14,0,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,3.39
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:12:00,FINGERS.SAUCED,2.0,0.985556,2,4,14,1,0.0,Wednesday,False,1,lunch,1,0,0,23.07,1.0,14.34
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:49:00,BROWNIE.COOKIE,2.0,0.985556,3,4,14,10,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,5.34


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - soda pop
2. 4 min later
    - meal
        - chicken fingers
3. 35 minutes after
    - dessert
        - brownie

**total drinks**: 1 non alc drink, **total meals**: 1 lunch meal, and 1 dessert.

**`lunch`**

`lunch_time`, `meal`, `dwell_time`

**Explanation**: classic lunch, even went for dessert. Lunches shouldn't usually dwell too long. no alcohol and a single meal.

**Features**: time of meal (1pm), no alcohol, and spent about an hour.

#### 4. Table 447381509

In [209]:
valentine[valentine.index == 434766693].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 00:50:00,PNT AMSTERDAM,1.0,1.294444,1,1,18,0,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 02:01:00,PNT MILL ST,1.0,1.294444,2,1,18,17,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86


**Feb 10th (Tuesday), 00:50 - 02:07, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - amsterdam beer, half liter
2. 1 hr and 10 min later
    - drink
        - mills, half liter

**total drinks**: 2 beer drinks

**`late night beer`** I think theres a difference between drinking late at night beer, and hard liquer.

    `time_between_drinks`, `beer`, `late_night` 

**Explanation**: pretty straight forward, late at night, drinking beer and taking its time. drinking a beer over an hour and then a second one.

**Features**: an hour between beers, having a total of two beers (seems like a common number of beers), and late night

#### 5. Table 446744761

In [210]:
valentine[valentine.index == 446799837].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:37:00,R.JT.MERLOT.Gl,6.0,1.335278,1,17,19,0,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,6.00
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:46:00,BACON.FILET*,2.0,1.335278,2,17,19,2,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,30.71


**Feb 12th (Tuesday), 18:37 - 19:57, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Wine
2. 10 min later
    - meal
        - Bacon Filet (huh?.. fancy)

**total drinks**: 1 drink, **total meals**: 1 dinner meal.

**`dinner`** (maybe even a celebratory dinner, or "pinuk" dinner? "treat yo self" type dinner.

`wine`, `expensive_meal`, `dinner_time`

**Explanation**: classic dinner table, although important to note that the wine and filet makes it a bit more fancy, special occasion type:)

**Features**: expensive meal -(30$), dinner time (6pm), `wine`.